In [1]:
# Install external library
!pip install pydub

In [2]:
# Import other libraries
import time
import requests
import numpy as np
import smtplib, ssl
import datetime as dt
from pydub import AudioSegment

In [5]:
# Request stream from Icecast server
streamurl = 'http://macalester-1.colo.iphouse.net:8080/wmcn'
r = requests.get(streamurl, stream=True)

# Define function that samples stream every other minute, then computes the 
# mean magnitude of the audiofile
def avg_stream_vol():
  with open('stream.mp3', 'wb') as f:
    try:
        for block in r.iter_content(1024):
          if dt.datetime.now().minute % 2 == 0:
            f.write(block)
          elif dt.datetime.now().minute % 2 != 0:
            time.sleep(60)
            break
          else:
            pass
    except KeyboardInterrupt:
        pass

  mp3 = AudioSegment.from_mp3('stream.mp3')
  raw = mp3.get_array_of_samples()
  raw = np.array(raw)
  raw = np.multiply(raw, raw)

  return raw.mean()

# Define function that takes output of avg_stream_vol(), determines if signal
# is live or dead, and sends email alert if signal is dead
def dead_air_alert(avg_vol):
  if avg_vol <= 1:
    dead = True
    alert = """\
    Subject: WMCN Signal Status

    Signal is dead. Red alert!""" + dt.datetime.now()
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, alert)
  else:
    pass

In [8]:
# Initialize parameters for silence sensor loop
dead = False
stop_date = dt.datetime(2021,8,14,0,0,0)
port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "jebcodes@gmail.com"
receiver_email = "jbeal@macalester.edu"
password = input("Type your password and press enter: ")

Type your password and press enter: testcodereceiver


In [9]:
# Run silence sensor
while dt.datetime.now() <= stop_date:
  avg_vol = avg_stream_vol()
  dead_air_alert(avg_vol)
  if dead:
    print('Signal is dead. Red alert!' + dt.datetime.now())
    break

CouldntDecodeError: ignored

Next steps:
- test and troubleshoot code under variety of circumstances and scenarios
- expand code to include a check for if the stream itself is functional (in case the url request fails because stream is down but signal may still be live)